In [1]:
import numpy as np

In [2]:
from helpers.replay_buffer import CountBasedReplayBuffer
from helpers.chain_environment import SimpleChain

In [3]:
reward_type = 'state'

In [4]:
replay_buffer = CountBasedReplayBuffer(1000, reward_type, lambda x: int(x.sum())-1, seed=42)

In [5]:
dim = 10
num_actions = 2
chain_env = SimpleChain(dim)

In [6]:
s_a_visitations = np.zeros((dim, num_actions))

In [15]:
eps = np.finfo(float).eps
def check_batch():
    batch_size = 32
    batch = replay_buffer.sample(batch_size, s_a_visitations)
    for i in range(batch_size):
        s = batch[0][i]
        s_id = int(s.sum())-1
        a = batch[1][i]
        r = batch[2][i]
        if reward_type == 'state':
            target_r = 1 / (np.sqrt(s_a_visitations[s_id].sum()) + 1)
        else:
            target_r = 1 / (np.sqrt(s_a_visitations[s_id][a]) + 1)
        #print(r, target_r)
        assert  r == target_r

In [16]:
for i in range(100):
    #print('episode {}'.format(i))
    s = chain_env.reset()
    while True:
        a = np.random.randint(num_actions)
        s_a_visitations[int(s.sum())-1][a] += 1 
        next_s, r, done, _ = chain_env.step(a)
        s = next_s
        replay_buffer.add(s, a, next_s, done)
        if done:
            break
    check_batch()

In [13]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [14]:
arr = np.arange(3)

In [15]:
softmax(arr)

array([0.09003057, 0.24472847, 0.66524096])

In [16]:
np.exp(arr) / np.exp(arr).sum()

array([0.09003057, 0.24472847, 0.66524096])